In [1]:
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

dataset = helpers.load_adult_income_dataset()
target = dataset["income"] # outcome variable
train_dataset, test_dataset, _, _ = train_test_split(dataset,
                                                     target,
                                                     test_size=0.2,
                                                     random_state=0,
                                                     stratify=target)

FileNotFoundError: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data not found.

In [ ]:
d = dice_ml.Data(dataframe=train_dataset,
                 continuous_features=['age', 'hours_per_week'],
                 outcome_name='income')

In [ ]:
m = dice_ml.Model(model_path=dice_ml.utils.helpers.get_adult_income_modelpath(),backend='TF2', func="ohe-min-max")

In [ ]:
exp = dice_ml.Dice(d,m)

In [ ]:
# Generate counterfactual examples
query_instance = test_dataset.drop(columns="income")[0:1]
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")
# Visualize counterfactual explanation
dice_exp.visualize_as_dataframe()

100%|██████████| 1/1 [00:01<00:00,  1.81s/it]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,Masters,Married,Blue-Collar,White,Female,38,1
1,29,Private,Prof-school,Married,Blue-Collar,White,Female,38,1
2,64.0,Private,Assoc,Married,Blue-Collar,White,Female,38,1
3,40.0,Private,HS-grad,Married,Professional,White,Female,38,1


In [27]:
imp = exp.local_feature_importance(query_instance, total_CFs=10)

100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


In [29]:
imp.local_importance

[{'hours_per_week': 0.6,
  'education': 0.5,
  'occupation': 0.3,
  'age': 0.3,
  'race': 0.1,
  'workclass': 0.0,
  'marital_status': 0.0,
  'gender': 0.0}]

In [1]:
# Sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# DiCE imports
import dice_ml
from dice_ml.utils import helpers  # helper functions

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [4]:
target = dataset["income"]
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

In [5]:
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

In [6]:
type(d)

dice_ml.data_interfaces.public_data_interface.PublicData

In [7]:
numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [8]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [9]:
e1 = exp.generate_counterfactuals(x_test[0:1], total_CFs=10, desired_class="opposite")
e1.visualize_as_dataframe()

100%|██████████| 1/1 [00:00<00:00,  1.63it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,Masters,Married,White-Collar,White,Female,38,1
1,29,Private,Assoc,Married,White-Collar,White,Female,38,1
2,29,Self-Employed,School,Married,Blue-Collar,White,Female,38,1
3,29,Private,Bachelors,Married,White-Collar,White,Female,38,1
4,29,Private,Masters,Married,Sales,White,Female,38,1
5,29,Private,Bachelors,Married,Other/Unknown,White,Female,38,1
6,29,Self-Employed,HS-grad,Married,White-Collar,White,Female,38,1
7,29,Private,Prof-school,Married,Blue-Collar,White,Male,38,1
8,29,Government,Prof-school,Married,Blue-Collar,White,Female,38,1
9,29,Self-Employed,HS-grad,Married,Professional,White,Female,38,1


In [61]:
class CasualExplanations:
    def __init__(self, exp, X, X_scaled, Y, categorical, numerical, classes, enc, ohe, x_min, x_max, rng, k=10, n_neighbor=10, **kwargs):
        self.exp = exp
        self.X = X
        self.X_scaled = X_scaled
        self.Y = Y
        self.categorical = categorical
        self.numerical = numerical
        self.classes = classes
        self.enc = enc
        self.ohe = ohe
        self.x_min = x_min
        self.x_max = x_max
        self.rng = rng
        self.k = k
        self.n_neighbor = n_neighbor
        if "low_limit" in kwargs:
            self.low_limit = kwargs["low_limit"]
        else:
            self.low_limit = 0
        if "high_limit" in kwargs:
            self.high_limit = kwargs["high_limit"]
        else:
            self.high_limit = len(X)
        self.cfs = self.generate_counterfactuals()
        self.neigh = self.k_neighbors()
        self.X_enc = self.enc.predict(self.X_scaled)
        self.class_mean = {}
        for c in classes:
            self.class_mean[c] = self.X_enc[self.Y.argmax(
                axis=1) == c].mean(axis=0)

    def generate_counterfactuals(self):
        return self.exp.generate_counterfactuals(self.X[self.low_limit:self.high_limit], total_CFs=self.k, desired_class="opposite")

    def k_neighbors(self):
        from sklearn.neighbors import NearestNeighbors
        neigh = NearestNeighbors(n_neighbors=self.n_neighbor)
        neigh.fit(self.X_scaled)
        return neigh

    def average_sparsity(self, id):
        sum_sparsity = 0
        for i in range(self.k):
            sum_sparsity += (
                self.cfs.cf_examples_list[id].final_cfs_df.iloc[i, :-1] != self.X.iloc[id]).sum()
        return sum_sparsity / self.k

    def total_average_sparsity(self):
        sum_sparsity = 0
        for i in range(self.low_limit, self.high_limit):
            sum_sparsity += self.average_sparsity(i)
        return sum_sparsity / (self.high_limit - self.low_limit)

    def average_proximity(self, id):
        sum_proximity = 0
        for i in range(self.k):
            proximity = 0
            for col in self.categorical:
                if self.cfs.cf_examples_list[id].final_cfs_df.iloc[i, :-1][col] != self.X.iloc[id][col]:
                    proximity += 1
            for col in self.numerical:
                proximity += abs(
                    self.cfs.cf_examples_list[id].final_cfs_df.iloc[i, :-1][col] - self.X.iloc[id][col])
            sum_proximity += proximity
        return sum_proximity / self.k

    def total_average_proximity(self):
        sum_proximity = 0
        for i in range(self.low_limit, self.high_limit):
            sum_proximity += self.average_proximity(i)
        return sum_proximity / (self.high_limit - self.low_limit)

    def average_diversity(self, id):
        sum_diversity = 0
        for i in range(self.k-1):
            diversity = 0
            for j in range(i+1, self.k):
                for col in self.categorical:
                    if self.cfs.cf_examples_list[id].final_cfs_df.iloc[i, :-1][col] != self.cfs.cf_examples_list[id].final_cfs_df.iloc[j, :-1][col]:
                        diversity += 1
                for col in self.numerical:
                    diversity += abs(self.cfs.cf_examples_list[id].final_cfs_df.iloc[i, :-1]
                                     [col] - self.cfs.cf_examples_list[id].final_cfs_df.iloc[j, :-1][col])
            sum_diversity += diversity
        return sum_diversity / (self.k**2)

    def total_average_diversity(self):
        sum_diversity = 0
        for i in range(self.low_limit, self.high_limit):
            sum_diversity += self.average_diversity(i)
        return sum_diversity / (self.high_limit - self.low_limit)

    def average_interpretability(self, id):
        x_cf = self.cfs.cf_examples_list[id].final_cfs_df.iloc[:, :-1]
        y_cf = self.cfs.cf_examples_list[id].final_cfs_df.iloc[:, -1]
        x_cf = np.c_[self.ohe.transform(x_cf.loc[:, self.categorical]), (x_cf.loc[:, self.numerical] - self.x_min) / (
            self.x_max - self.x_min) * (self.rng[1] - self.rng[0]) + self.rng[0]].astype(np.float32, copy=False)
        cf_enc = self.enc.predict(x_cf)
        sum_interpretability = 0
        for i in range(self.k):
            dist_orig = np.linalg.norm(
                cf_enc[i] - self.class_mean[self.Y[id].argmax(axis=0)])
            dist_cf = np.linalg.norm(cf_enc[i] - self.class_mean[y_cf[i]])
            sum_interpretability += dist_orig / (dist_cf + 1e-10)
        return sum_interpretability / self.k

    def total_average_interpretability(self):
        sum_interpretability = 0
        for i in range(self.low_limit, self.high_limit):
            sum_interpretability += self.average_interpretability(i)
        return sum_interpretability / (self.high_limit - self.low_limit)

    def average_interpretability_v2(self, id):
        sum_interpretability = 0
        for i in range(self.k):
            interpretability = 0
            for col in self.categorical:
                if self.cfs.cf_examples_list[id].final_cfs_df.iloc[i, :-1][col] != self.X.iloc[id][col]:
                    interpretability += 1
            for col in self.numerical:
                interpretability += abs(self.cfs.cf_examples_list[id].final_cfs_df.iloc[i, :-1][col] - self.X.iloc[id][col])/(
                    max(self.X.loc[:, col]) - min(self.X.loc[:, col]))
            sum_interpretability += interpretability / \
                (len(self.categorical) + len(self.numerical))
        return sum_interpretability / self.k

    def total_average_interpretability_v2(self):
        sum_interpretability = 0
        for i in range(self.low_limit, self.high_limit):
            sum_interpretability += self.average_interpretability_v2(i)
        return sum_interpretability / (self.high_limit - self.low_limit)

    def average_proximity_between_labels(self, id):
        sum_proximity = 0
        x_cf = self.cfs.cf_examples_list[id].final_cfs_df.iloc[:, :-1]
        x_cf = np.c_[self.ohe.transform(x_cf.loc[:, self.categorical]), (x_cf.loc[:, self.numerical] - self.x_min) / (
            self.x_max - self.x_min) * (self.rng[1] - self.rng[0]) + self.rng[0]].astype(np.float32, copy=False)
        for i in range(self.k):
            proximity = 0
            k_neighbors = self.neigh.kneighbors(x_cf[i].reshape(1, -1))
            p_sum = 0
            for j in range(self.n_neighbor):
                p_sum += k_neighbors[0][0][j]
            for j in range(self.n_neighbor):
                proximity += (k_neighbors[0][0][j]*k_neighbors[1][0][j])/p_sum
            sum_proximity += proximity/x_cf.shape[-1]
        return sum_proximity / self.k
    
    def total_average_proximity_between_labels(self):
        sum_proximity = 0
        for i in range(self.low_limit, self.high_limit):
            sum_proximity += self.average_proximity_between_labels(i)
        return sum_proximity / (self.high_limit - self.low_limit)
            

In [19]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical

In [20]:
import numpy as np

In [21]:
x_train_num = x_train.loc[:, numerical].astype(np.float32, copy=False)

In [22]:
x_train_cat = x_train.loc[:, categorical].copy()

In [23]:
x_min, x_max = x_train_num.min(axis=0), x_train_num.max(axis=0)

In [24]:
rng = (-1., 1.)
X_num_scaled = (x_train_num - x_min) / (x_max - x_min) * (rng[1] - rng[0]) + rng[0]

In [71]:
X_num_scaled

,age,hours_per_week
20907,-0.424658,-0.122449
2573,0.013699,0.204082
10939,-0.616438,-0.204082
7839,-0.342466,-0.204082
9608,-0.726027,-0.204082
...,...,...
25388,-0.041096,-0.612245
6214,-0.397260,-0.204082
6749,-0.260274,0.102041
17884,-0.835616,-0.204082


In [35]:
ohe = OneHotEncoder(categories='auto', sparse=False).fit(x_train_cat)
X_cat_ohe = ohe.transform(x_train_cat)

d:\Anaconda\envs\interpret\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [36]:
X_train = np.c_[X_cat_ohe, X_num_scaled].astype(np.float32, copy=False)

In [37]:
X_test = np.c_[ohe.transform(x_test.loc[:, categorical]), (x_test.loc[:, numerical] - x_min) / (x_max - x_min) * (rng[1] - rng[0]) + rng[0]].astype(np.float32, copy=False)

In [38]:
X_train.shape, X_test.shape

((20838, 29), (5210, 29))

In [39]:
X_train.shape[-1]

29

In [40]:
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

In [41]:
def ae_model():
    # encoder
    x_in = Input(shape=(29,))
    x = Dense(60, activation='relu')(x_in)
    x = Dense(30, activation='relu')(x)
    x = Dense(15, activation='relu')(x)
    encoded = Dense(10, activation=None)(x)
    encoder = Model(x_in, encoded)
    
    # decoder
    dec_in = Input(shape=(10,))
    x = Dense(15, activation='relu')(dec_in)
    x = Dense(30, activation='relu')(x)
    x = Dense(60, activation='relu')(x)
    decoded = Dense(29, activation=None)(x)
    decoder = Model(dec_in, decoded)
    
    # autoencoder = encoder + decoder
    x_out = decoder(encoder(x_in))
    autoencoder = Model(x_in, x_out)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder, encoder, decoder

In [42]:
ae, enc, dec = ae_model()
ae.summary()
ae.fit(X_train, X_train, batch_size=128, epochs=100, validation_data=(X_test, X_test), verbose=1)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 29)]              0         
                                                                 
 model_6 (Functional)        (None, 10)                4255      
                                                                 
 model_7 (Functional)        (None, 29)                4274      
                                                                 
Total params: 8,529
Trainable params: 8,529
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
163/163 [==============================] - 2s 5ms/step - loss: 0.1030 - val_loss: 0.0651
Epoch 2/100
163/163 [==============================] - 1s 4ms/step - loss: 0.0520 - val_loss: 0.0413
Epoch 3/100
163/163 [==============================] - 1s 4ms/step - loss: 0.0341 - val_loss: 0.0285
Epoch 4/100
163/16

KeyboardInterrupt: 

In [ ]:
cf = e1.cf_examples_list[0].final_cfs_df

In [ ]:
x_cf = cf.iloc[:, :-1].copy()

In [ ]:
xf_cf = np.c_[ohe.transform(x_cf.loc[:, categorical]), (x_cf.loc[:, numerical] - x_min) / (x_max - x_min) * (rng[1] - rng[0]) + rng[0]].astype(np.float32, copy=False)

In [ ]:
enc.predict(xf_cf)

1/1 [==============================] - 0s 19ms/step


array([[-4.2247496 , -1.0174646 , -2.2552083 ,  1.0896508 , -3.8075192 ,
         3.5094485 ,  4.9353867 , -3.3043954 , -1.3877498 ,  3.4297454 ],
       [-2.934661  ,  1.2704629 , -2.3813436 ,  1.2012497 , -3.2195582 ,
         2.0804317 ,  5.318864  , -2.4021032 , -0.23903197,  4.0691113 ],
       [-3.9927645 ,  0.5354843 , -1.4090974 , -1.6454202 , -4.23142   ,
         4.1528335 ,  4.4369283 , -2.8486516 , -0.3754536 ,  2.2261927 ],
       [-6.7905207 ,  0.29155096, -1.7372558 ,  0.28913617, -4.773374  ,
         3.72803   ,  3.6087556 , -3.5164623 ,  0.6605272 ,  0.33095303],
       [-6.332353  ,  0.8971953 , -1.8257024 ,  0.24094963, -4.040234  ,
         4.449658  ,  4.5445    , -3.096514  ,  1.0572677 ,  0.9613028 ],
       [-6.2996807 ,  0.23256153, -2.3017726 ,  0.75853693, -3.9724278 ,
         3.7458959 ,  3.6445465 , -2.8685026 ,  0.9197155 ,  1.9694383 ],
       [-5.5311027 , -0.3348734 , -2.7802815 ,  2.7801147 , -2.6662118 ,
         2.136769  ,  3.41835   , -3.6901987 

In [ ]:
X_enc = enc.predict(X_train)

652/652 [==============================] - 1s 2ms/step


In [ ]:
Y_train.argmax(axis=1).shape

(20838,)

In [ ]:
# class wise mean of encoded features
class_mean = {}
class_mean[0] = X_enc[Y_train.argmax(axis=1)==0].mean(axis=0)
class_mean[1] = X_enc[Y_train.argmax(axis=1)==1].mean(axis=0)

In [ ]:
cf_enc = enc.predict(xf_cf)

1/1 [==============================] - 0s 21ms/step


In [114]:
sum_inter = 0
for i in cf_enc:
    dist_orig = np.linalg.norm(i - class_mean[0])
    dist_adv = np.linalg.norm(i - class_mean[1])
    sum_inter += dist_orig / (dist_adv + 10e-10)

In [115]:
sum_inter / cf_enc.shape[0]

1.0966775561805018

In [62]:
def casual_interpret_full(train: tuple, test: tuple, exp: dice_ml.explainer_interfaces.dice_random.DiceRandom, categorical : list, numerical : list, classes : list, k : int, **kwargs):
    x_train, y_train = train
    x_test, y_test = test
    x_train_num = x_train.loc[:, numerical].astype(np.float32, copy=False)
    x_train_cat = x_train.loc[:, categorical].copy()
    x_min, x_max = x_train_num.min(axis=0), x_train_num.max(axis=0)
    rng = (-1., 1.)
    ohe = OneHotEncoder(categories='auto', sparse=False).fit(x_train_cat)
    X_train = np.c_[ohe.transform(x_train_cat), (x_train_num - x_min) / (x_max - x_min) * (rng[1] - rng[0]) + rng[0]].astype(np.float32, copy=False)
    X_test = np.c_[ohe.transform(x_test.loc[:, categorical]), (x_test.loc[:, numerical] - x_min) / (x_max - x_min) * (rng[1] - rng[0]) + rng[0]].astype(np.float32, copy=False)
    Y_train = to_categorical(y_train)
    Y_test = to_categorical(y_test)
    def ae_model():
        # encoder``
        x_in = Input(shape=(29,))
        x = Dense(60, activation='relu')(x_in)
        x = Dense(30, activation='relu')(x)
        x = Dense(15, activation='relu')(x)
        encoded = Dense(10, activation=None)(x)
        encoder = Model(x_in, encoded)
        # decoder
        dec_in = Input(shape=(10,))
        x = Dense(15, activation='relu')(dec_in)
        x = Dense(30, activation='relu')(x)
        x = Dense(60, activation='relu')(x)
        decoded = Dense(29, activation=None)(x)
        decoder = Model(dec_in, decoded)
        # autoencoder = encoder + decoder
        x_out = decoder(encoder(x_in))
        autoencoder = Model(x_in, x_out)
        autoencoder.compile(optimizer='adam', loss='mse')
        return autoencoder, encoder, decoder
    ae, enc, dec = ae_model()
    ae.fit(X_train, X_train, batch_size=128, epochs=10, validation_data=(X_test, X_test), verbose=1)
    train_ce = CasualExplanations(exp, x_train, X_train, Y_train, categorical, numerical, classes, enc, ohe, x_min, x_max, rng, k, **kwargs)
    test_ce = CasualExplanations(exp, x_test, X_test, Y_test, categorical, numerical, classes, enc, ohe, x_min, x_max, rng, k, **kwargs)
    return train_ce, test_ce

In [63]:
train, test = casual_interpret_full((x_train, y_train), (x_test, y_test), exp, categorical, numerical, [0,1], k=10, low_limit=0, high_limit=5)

d:\Anaconda\envs\interpret\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
163/163 [==============================] - 2s 5ms/step - loss: 0.0988 - val_loss: 0.0616
Epoch 2/10
163/163 [==============================] - 1s 4ms/step - loss: 0.0479 - val_loss: 0.0370
Epoch 3/10
163/163 [==============================] - 0s 2ms/step - loss: 0.0310 - val_loss: 0.0268
Epoch 4/10
163/163 [==============================] - 0s 2ms/step - loss: 0.0239 - val_loss: 0.0219
Epoch 5/10
163/163 [==============================] - 1s 4ms/step - loss: 0.0202 - val_loss: 0.0191
Epoch 6/10
163/163 [==============================] - 1s 4ms/step - loss: 0.0179 - val_loss: 0.0171
Epoch 7/10
163/163 [==============================] - 1s 4ms/step - loss: 0.0161 - val_loss: 0.0156
Epoch 8/10
163/163 [==============================] - 1s 4ms/step - loss: 0.0148 - val_loss: 0.0145
Epoch 9/10
163/163 [==============================] - 1s 4ms/step - loss: 0.0137 - val_loss: 0.0134
Epoch 10/10
163/163 [==============================] - 1s 4ms/step - loss: 0.0128 - val_loss: 0.0126

100%|██████████| 5/5 [00:03<00:00,  1.51it/s]

 78/652 [==>...........................] - ETA: 0s

652/652 [==============================] - 1s 1ms/step


100%|██████████| 5/5 [00:02<00:00,  1.82it/s]

 77/163 [=============>................] - ETA: 0s

163/163 [==============================] - 0s 2ms/step


In [64]:
train.total_average_sparsity(), test.total_average_sparsity()

(2.16, 2.12)

In [65]:
train.total_average_proximity(), test.total_average_proximity()

(17.72, 8.02)

In [66]:
train.total_average_diversity(), test.total_average_diversity()

(13.540000000000001, 6.3260000000000005)

In [67]:
train.total_average_interpretability(), test.total_average_interpretability()

1/1 [==============================] - 0s 50ms/step


(1.0051886650549895, 1.1345499695099577)

In [68]:
train.total_average_interpretability_v2(), test.total_average_interpretability_v2()

(0.2142189684092815, 0.23437517472742525)

In [69]:
train.total_average_proximity_between_labels(), test.total_average_proximity_between_labels()

(356.2896740772791, 83.67289459096745)